In [1]:
import nltk
from gensim.models import Word2Vec
from gensim.models import KeyedVectors
from nltk.corpus import stopwords
from nltk import word_tokenize
from nltk.corpus import stopwords
from unidecode import unidecode
import string
import numpy as np
import re
from collections import Counter
import itertools
from sklearn.metrics.pairwise import cosine_similarity

In [11]:
paragraph = """I have three visions for India. In 3000 years of our history, people from all over 
               the world have come and invaded us, captured our lands, conquered our minds. 
               From Alexander onwards, the Greeks, the Turks, the Moguls, the Portuguese, the British,
               the French, the Dutch, all of them came and looted us, took over what was ours. 
               Yet we have not done this to any other nation. We have not conquered anyone. 
               We have not grabbed their land, their culture, 
               their history and tried to enforce our way of life on them. 
               Why? Because we respect the freedom of others.That is why my 
               first vision is that of freedom. I believe that India got its first vision of 
               this in 1857, when we started the War of Independence. It is this freedom that
               we must protect and nurture and build on. If we are not free, no one will respect us.
               My second vision for India’s development. For fifty years we have been a developing nation.
               It is time we see ourselves as a developed nation. We are among the top 5 nations of the world
               in terms of GDP. We have a 10 percent growth rate in most areas. Our poverty levels are falling.
               Our achievements are being globally recognised today. Yet we lack the self-confidence to
               see ourselves as a developed nation, self-reliant and self-assured. Isn’t this incorrect?
               I have a third vision. India must stand up to the world. Because I believe that unless India 
               stands up to the world, no one will respect us. Only strength respects strength. We must be 
               strong not only as a military power but also as an economic power. Both must go hand-in-hand. 
               My good fortune was to have worked with three great minds. Dr. Vikram Sarabhai of the Dept. of 
               space, Professor Satish Dhawan, who succeeded him and Dr. Brahm Prakash, father of nuclear material.
               I was lucky to have worked with all three of them closely and consider this the great opportunity of my life. 
               I see four milestones in my career"""

In [2]:
def preprocess_text(text):
    text = re.sub(r'\[[0-9]*\]',' ',paragraph)
    text = re.sub(r'\s+',' ',text)
    text = text.lower()
    text = re.sub(r'\d',' ',text)
    text = re.sub(r'\s+',' ',text)
    stopset = stopwords.words('english') + list(string.punctuation)
    text = " ".join([i for i in word_tokenize(text) if i not in stopset])
    text = unidecode(text)
    return text

In [25]:
paragraph = preprocess_text(paragraph)

"three visions india years history people world come invaded us captured lands conquered minds alexander onwards greeks turks moguls portuguese british french dutch came looted us took yet done nation conquered anyone grabbed land culture history tried enforce way life respect freedom others.that first vision freedom believe india got first vision started war independence freedom must protect nurture build free one respect us second vision india ' development fifty years developing nation time see developed nation among top nations world terms gdp percent growth rate areas poverty levels falling achievements globally recognised today yet lack self-confidence see developed nation self-reliant self-assured ' incorrect third vision india must stand world believe unless india stands world one respect us strength respects strength must strong military power also economic power must go hand-in-hand good fortune worked three great minds dr. vikram sarabhai dept space professor satish dhawan s

In [3]:
#word2vec-google-news-300(1.5GB)
#glove-twitter-200 (758 MB)
import gensim.downloader as api
print(api.load("glove-twitter-25", return_path=True))

[==================================================] 100.0% 104.8/104.8MB downloaded
/home/abhrant/gensim-data/glove-twitter-25/glove-twitter-25.gz


In [27]:
word_emb_model = KeyedVectors.load_word2vec_format("glove-twitter-200.gz")

In [8]:
def map_word_frequency(document):
    return Counter(itertools.chain(*document))

In [13]:
def get_sif_feature_vectors(sentence1, sentence2, word_emb_model=word_emb_model):
    sentence1 = [token for token in sentence1.split() if token in word_emb_model.wv.vocab]
    sentence2 = [token for token in sentence2.split() if token in word_emb_model.wv.vocab]
    word_counts = map_word_frequency((sentence1 + sentence2))
    embedding_size = 200
    a = 0.001
    sentence_set=[]
    for sentence in [sentence1, sentence2]:
        vs = np.zeros(embedding_size)
        sentence_length = len(sentence)
        for word in sentence:
            a_value = a / (a + word_counts[word]) 
            vs = np.add(vs, np.multiply(a_value, word_emb_model.wv[word])) 
        vs = np.divide(vs, sentence_length) 
        sentence_set.append(vs)
    return sentence_set

In [14]:
def get_cosine_similarity(feature_vec_1, feature_vec_2):    
    return cosine_similarity(feature_vec_1.reshape(1, -1), feature_vec_2.reshape(1, -1))[0][0]

In [24]:
sent_1 = "Doraemon is a cartoon"
sent_2 = "cartoon"

In [25]:
sent_set=get_sif_feature_vectors(sent_1,sent_2,word_emb_model)

/home/abhrant/python/deep_learning/lib/python3.6/site-packages/ipykernel_launcher.py:2: DeprecationWarning: Call to deprecated `wv` (Attribute will be removed in 4.0.0, use self instead).
  
/home/abhrant/python/deep_learning/lib/python3.6/site-packages/ipykernel_launcher.py:3: DeprecationWarning: Call to deprecated `wv` (Attribute will be removed in 4.0.0, use self instead).
  This is separate from the ipykernel package so we can avoid doing imports until
/home/abhrant/python/deep_learning/lib/python3.6/site-packages/ipykernel_launcher.py:13: DeprecationWarning: Call to deprecated `wv` (Attribute will be removed in 4.0.0, use self instead).
  del sys.path[0]


In [26]:
get_cosine_similarity(sent_set[0],sent_set[1])

0.7869705842225252